In [1]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm.session import sessionmaker
from sqlalchemy import inspect
from sqlalchemy.sql import func
import geoalchemy2
from sqlalchemy import Table, MetaData
from sqlalchemy.ext.declarative import declarative_base

In [2]:
def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(
    user=POSTGRES_USER,
    pw=POSTGRES_PW,
    url=POSTGRES_URL,
    db=POSTGRES_DB
)

In [3]:
engine = create_engine(DB_URL)
sqlahelper.add_engine(engine, 'se4all_engine')

## Make a query 

In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [18]:
# Pure SQL command
res = session.query("""SUM(length_km) FROM se4all.distribution_line_se4all;""")
res.all()

In [ ]:
res = session.query("""schemaname as schema_name, matviewname as view_name, matviewowner as owner, ispopulated as is_populated, definition FROM pg_matviews;""")
res.all()

In [37]:
with engine.connect() as con:
    
    #rs = con.execute('SELECT * FROM se4all.distribution_line_all_mv;')
    rs = con.execute("""SELECT oid::regclass::text
FROM   pg_class
WHERE  relkind = 'm';""")
    
    data = rs.fetchall()
    print(data)

[('se4all.cluster_offgrid_ng003_mv',), ('se4all.join_osm_building_adm1_buffer_mv',), ('se4all.cluster_offgrid_ng005_mv',), ('se4all.cluster_offgrid_ng011_mv',), ('se4all.join_distribution_line_all_buffer_adm1_mv',), ('se4all.cluster_offgrid_ng001_mv',), ('se4all.join_osm_building_distribution_line_all_buffer_mv',), ('se4all.join_cluster_all_adm1_mv',), ('se4all.cluster_offgrid_ng020_mv',), ('se4all.cluster_offgrid_ng034_mv',), ('se4all.cluster_offgrid_ng021_mv',), ('se4all.cluster_offgrid_ng016_mv',), ('se4all.cluster_offgrid_ng025_mv',), ('se4all.cluster_offgrid_ng029_mv',), ('se4all.cluster_offgrid_ng018_mv',), ('se4all.cluster_offgrid_ng006_mv',), ('se4all.cluster_offgrid_ng002_mv',), ('se4all.cluster_offgrid_ng004_mv',), ('se4all.cluster_offgrid_ng008_mv',), ('se4all.boundary_adm1_buffer_mv',), ('se4all.distribution_line_all_mv',), ('se4all.cluster_offgrid_ng014_mv',), ('se4all.cluster_offgrid_ng012_mv',), ('se4all.cluster_offgrid_ng023_mv',), ('se4all.cluster_offgrid_ng030_mv',), 

In [56]:
def list_materialized_view(engine):
    views = []
    with engine.connect() as con:
    
        rs = con.execute("""SELECT oid::regclass::text FROM   pg_class WHERE  relkind = 'm';""")
    
        for r in rs.fetchall():
            views.append(r[0])
    return views

def select_materialized_view(engine, view_name, schema=None, limit=None):
    if schema is not None:
        view_name = "{}.{}".format(schema, view_name)
    if limit is None:
        limit = ""
    else:
        limit = " LIMIT {}".format(limit)
    with engine.connect() as con:
        rs = con.execute('SELECT * FROM {}{};'.format(view_name, limit))
        data = rs.fetchall()
    return data.geom
select_materialized_view(engine, "se4all.distribution_line_all_mv", limit=10)

AttributeError: 'list' object has no attribute 'geom'

In [ ]:
with engine.connect() as con:
    rs.con.execute("""SELECT a.attname,
       pg_catalog.format_type(a.atttypid, a.atttypmod),
       a.attnotnull
FROM pg_attribute a
  JOIN pg_class t on a.attrelid = t.oid
  JOIN pg_namespace s on t.relnamespace = s.oid
WHERE a.attnum > 0 
  AND NOT a.attisdropped
  AND t.relname = 'mv_name' --<< replace with the name of the MV 
  AND s.nspname = 'public' --<< change to the schema your MV is in 
ORDER BY a.attnum;""")
    print(rs.fetchall())

In [25]:
# Same command in SQlAlchemy
metadata = MetaData(schema='se4all', bind=engine)

Base = declarative_base(metadata=metadata)

class DlinesSe4all(Base):
    __table__ = Table('distribution_line_se4all', Base.metadata, autoload=True, autoload_with=engine)
    
res = session.query(func.sum(DlinesSe4all.length_km))
res.all()

[(Decimal('16409.268'))]

## Look at the database

In [5]:
inspector = inspect(engine)
inspector.get_schema_names()

['geonode', 'import', 'information_schema', 'public', 'se4all']

In [6]:
inspector.get_table_names('se4all')

['data_version',
 'database_log',
 'boundary_adm1',
 'distribution_line_kaduna_electric',
 'cluster_all',
 'cluster_offgrid',
 'distribution_line_kedco',
 'osm_building',
 'distribution_line_se4all',
 'boundary_adm1_status']

In [ ]:
inspector.get_columns('database_log', 'se4all')

In [9]:
inspector.get_view_names()

['pg_stat_statements',
 'pg_buffercache',
 'geography_columns',
 'geometry_columns',
 'raster_columns',
 'raster_overviews']